In [1]:
# Thiết lập môi trường và import các hàm cần thiết
import numpy as np
import sys
import os

# Thiết lập đường dẫn gốc của project
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
os.chdir(project_root)
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
# Import các hàm xử lý từ module data_processing
from src.data_processing import (
    load_data,
    handle_missing_categorical,
    encode_ordinal_features,
    encode_one_hot_features,
    process_and_scale_numerical,
    create_new_features,
    handle_outliers_iqr
)
from src.models import save_model

print("Các hàm cần thiết từ `src` đã được import.")

[2025-11-18 20:15:11,946] [INFO] [src] - SRC package phiên bản 1.0.0 đã được khởi tạo
Các hàm cần thiết từ `src` đã được import.


In [2]:
# Tải dữ liệu thô
file_path = 'data/raw/aug_test.csv'
data = load_data(file_path)

[2025-11-18 20:15:11,993] [INFO] [src.data_processing] - Tải dữ liệu thành công từ 'data/raw/aug_test.csv'.


Xử lý các giá trị ngoại lai trong cột `training_hours` được xác định ở bước EDA.
- **Phương pháp:** Sử dụng Interquartile Range (IQR).
- **Hành động:** `clip` (giới hạn) các giá trị nằm ngoài khoảng `[Q1 - 1.5*IQR, Q3 + 1.5*IQR]`.
- **Kết quả:** Các giá trị quá lớn được đưa về một ngưỡng tối đa, giúp giảm độ lệch của dữ liệu nhưng vẫn giữ nguyên toàn bộ số dòng.

In [3]:
# Xử lý outliers cho training_hours
if data.size > 0:
    # Sử dụng phương pháp 'clip'
    data_no_outliers = handle_outliers_iqr(data, 'training_hours', method='clip')
    
    print(f"Số lượng dòng ban đầu: {len(data)}")
    print(f"Số lượng dòng sau khi xử lý outliers: {len(data_no_outliers)}")
    # Giá trị lớn nhất mới sẽ là ngưỡng upper_bound
    print(f"Giá trị training_hours lớn nhất sau khi clip: {np.max(data_no_outliers['training_hours'])}")

Số lượng dòng ban đầu: 2129
Số lượng dòng sau khi xử lý outliers: 2129
Giá trị training_hours lớn nhất sau khi clip: 180


In [4]:
# Bước 1: tạo đặc trưng mới (Feature Engineering)
if data.size > 0:
    data_featured = create_new_features(data)

[2025-11-18 20:15:12,030] [INFO] [src.data_processing] - Bắt đầu tạo đặc trưng mới...
[2025-11-18 20:15:12,035] [INFO] [src.data_processing] - Hoàn tất Feature Engineering.


In [5]:
# Bước 2: xử lý giá trị thiếu
if 'data_featured' in locals():
    categorical_cols_with_missing = [
        'gender', 'enrolled_university', 'education_level', 
        'major_discipline', 'company_size', 'company_type'
    ]
    data_imputed = handle_missing_categorical(data_featured, categorical_cols_with_missing)

[2025-11-18 20:15:12,051] [INFO] [src.data_processing] - Hoàn tất xử lý giá trị thiếu cho các cột categorical.


In [6]:
# Bước 3: mã hóa các cột có thứ tự (Ordinal Encoding)
if 'data_imputed' in locals():
    data_ordinal, ordinal_mappings = encode_ordinal_features(data_imputed)

[2025-11-18 20:15:12,070] [INFO] [src.data_processing] - Hoàn tất mã hóa ordinal.


In [7]:
# Bước 4: mã hóa các cột không có thứ tự (One-Hot Encoding)
if 'data_ordinal' in locals():
    nominal_cols = [
        'city', 'gender', 'enrolled_university', 
        'major_discipline', 'company_type', 'last_new_job'
    ]
    one_hot_matrix, one_hot_mappings = encode_one_hot_features(data_ordinal, nominal_cols)

[2025-11-18 20:15:12,089] [INFO] [src.data_processing] - Đã mã hóa 'city' thành 108 features.
[2025-11-18 20:15:12,090] [INFO] [src.data_processing] - Đã mã hóa 'gender' thành 4 features.
[2025-11-18 20:15:12,092] [INFO] [src.data_processing] - Đã mã hóa 'enrolled_university' thành 4 features.
[2025-11-18 20:15:12,094] [INFO] [src.data_processing] - Đã mã hóa 'major_discipline' thành 7 features.
[2025-11-18 20:15:12,097] [INFO] [src.data_processing] - Đã mã hóa 'company_type' thành 7 features.
[2025-11-18 20:15:12,098] [INFO] [src.data_processing] - Đã mã hóa 'last_new_job' thành 7 features.
[2025-11-18 20:15:12,103] [INFO] [src.data_processing] - Hoàn tất mã hóa one-hot. Ma trận cuối cùng có shape: (2129, 137)


In [8]:
# Bước 5: xử lý và chuẩn hóa các cột số (Numerical Scaling)
if 'data_ordinal' in locals():
    data_scaled, scaling_params = process_and_scale_numerical(data_ordinal)

[2025-11-18 20:15:12,120] [INFO] [src.data_processing] - Hoàn tất xử lý và chuẩn hóa các cột số.


In [9]:
# Bước 6: tổng hợp tất cả các features thành một ma trận cuối cùng
if 'data_scaled' in locals() and 'one_hot_matrix' in locals():
    # Lấy ra các cột số đã chuẩn hóa
    numerical_features = np.array(data_scaled[['city_development_index', 'training_hours', 'experience']].tolist())
    # Lấy ra các cột thứ tự đã mã hóa
    ordinal_features = np.array(data_scaled[['relevent_experience', 'education_level', 'company_size']].tolist())

    # Ghép tất cả lại: Cột số + Cột thứ tự + Cột one-hot
    X_test_processed = np.hstack([numerical_features, ordinal_features, one_hot_matrix])
    
    print(f"Ma trận features cuối cùng được tạo với shape: {X_test_processed.shape}")

Ma trận features cuối cùng được tạo với shape: (2129, 143)


In [10]:
# Bước 7: lưu dữ liệu đã xử lý và các artifacts
if 'X_test_processed' in locals():
    processed_matrix_path = 'data/processed/X_test_processed.npy'
    processed_csv_path = 'data/processed/processed_dataset.csv'
    artifacts_path = 'models/preprocessor_artifacts.pkl'
    
    # Lưu ma trận features
    np.save(processed_matrix_path, X_test_processed)
    print(f"Đã lưu ma trận features vào '{processed_matrix_path}'")
    csv_header = ','.join(data_scaled.dtype.names)

    np.savetxt(
        processed_csv_path, 
        data_scaled, 
        delimiter=',',
        fmt='%s',
        header=csv_header,
        encoding='utf-8',
        comments=''
    )
    print(f"Đã lưu bộ dữ liệu đã xử lý vào '{processed_csv_path}'")

    # Gộp và lưu các artifacts (mappings, params) để sử dụng sau này
    preprocessor_artifacts = {
        'ordinal_mappings': ordinal_mappings,
        'one_hot_mappings': one_hot_mappings,
        'scaling_params': scaling_params
    }
    save_model(preprocessor_artifacts, artifacts_path)

Đã lưu ma trận features vào 'data/processed/X_test_processed.npy'


Đã lưu bộ dữ liệu đã xử lý vào 'data/processed/processed_dataset.csv'
[2025-11-18 20:15:12,315] [INFO] [src.models] - Lưu model thành công vào 'models/preprocessor_artifacts.pkl'.
